### build-mysql-database
CONSTROI BASE DE DADOS NO MYSQL POR MEIO DE DOCUMENTOS DA COLEÇÃO **ANTONIO AZEREDO DA SILVEIRA, MINISTÉRIO DAS RELAÇÕES EXTERIORES**.  

AS INFORMAÇÕES DE CADA DOCUMENTO A SEREM INSERIDAS NA BASE SÃO:
* ID
* CORPO DE TEXTO
* DATAS (ANO E MÊS)
* URL
* IDIOMA PRINCIPAL

In [1]:
import os
import sys
import time
import datetime
import numpy as np
import pandas as pd
import re
import pymysql
#import psycopg2
import getpass

from sshtunnel import SSHTunnelForwarder
import time

import codecs
from langdetect import detect
from IPython.display import clear_output
import csv

In [2]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
define pastas de trabalho
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

#path = "C:/Users/marcelo.ribeiro/Documents/textfiles-corrected-regrouped/"
#path = "C:/Users/marcelo.ribeiro/Documents/temp/" #TESTE!
#outputs = "C:/Users/marcelo.ribeiro/Documents/outputs/"
path = "/home/rsouza/Documentos/text-learning-tools/textfiles-corrected-regrouped/"

files = [f for f in os.listdir(path)]
fullpath_list = []
fullpath = ''
for file in files:
    fullpath = path+file
    fullpath_list.append(fullpath)

In [3]:
files[0:4]

['AAS_mre_rb_1974.04.17_doc_I-66.txt',
 'AAS_mre_onu_1974.03.12_doc_II-49.txt',
 'AAS_mre_onu_1974.03.12_doc_V-2.txt',
 'AAS_mre_be_1974.04.16_doc_VI-18.txt']

In [7]:
fullpath_list[0:4]

['/home/rsouza/Documentos/text-learning-tools/textfiles-corrected-regrouped/AAS_mre_rb_1974.04.17_doc_I-66.txt',
 '/home/rsouza/Documentos/text-learning-tools/textfiles-corrected-regrouped/AAS_mre_onu_1974.03.12_doc_II-49.txt',
 '/home/rsouza/Documentos/text-learning-tools/textfiles-corrected-regrouped/AAS_mre_onu_1974.03.12_doc_V-2.txt',
 '/home/rsouza/Documentos/text-learning-tools/textfiles-corrected-regrouped/AAS_mre_be_1974.04.16_doc_VI-18.txt']

In [5]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
Cria listas de url e dossies correspondentes
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
url_list = []
dossie_list = []
#url_inputs = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Azeredo Papers/Processamento/text-learning-tools/inputs/URLS-AAS_2.csv"
url_inputs = "/home/rsouza/Documentos/text-learning-tools/inputs/URLS-AAS_2.csv"
with open(url_inputs, 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    for row in reader:
        if ' mre ' not in row[1]: continue
        dossie = re.sub(' ','_',row[1])
        dossie = re.sub('AAS_mre_(.*)',r'\1',dossie)
        dossie_list.append(dossie)
        url_list.append(row[2])

In [11]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
Cria e alimenta a base mysql
Classifica o corpus de acordo com: tipo textual (para separar textos sujos, como letra cursiva, foto, fonte alternativa) e idioma

Textos ilegíveis são retirados da base mysql, mas listados na pasta Outputs para que busquemos soluções futuramente.

pt = 1
en = 2
es = 3
fr = 4
de = 5
não identificado = 0
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
count = 0
temp = 0

doc_class = []
lang_class = 0
count_doc = 0
not_readable = []
percentil = int(len(fullpath_list)/100)

pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()
with SSHTunnelForwarder(('200.20.164.147', 22),
                        #ssh_private_key = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa",
                        ssh_private_key = '/home/rsouza/.ssh/id_rsa',
                        ssh_private_key_password = pass_sshkey,
                        #ssh_username="marcelobribeiro",
                        ssh_username="rsouza",
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           #user='marcelobribeiro', 
                           user='rsouza',
                           passwd=pass_mysql,
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    insere dados na base mysql
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    
    cur.execute("DROP TABLE doc")
    cur.execute('''CREATE TABLE IF NOT EXISTS doc 
               (id VARCHAR(50), body LONGTEXT, year VARCHAR(4), month VARCHAR(2), url VARCHAR(200), main_language VARCHAR(10), temp VARCHAR(2)
               DEFAULT NULL)
               ENGINE=MyISAM DEFAULT CHARSET='utf8';''')
    #cur.execute("DELETE FROM doc WHERE temp = 0")
    
    for txt in fullpath_list[0:200]:
        
        ### mede percentual de conclusão da tarefa ###
        count_doc += 1
        if count_doc % percentil == 0: print(int(count_doc/percentil),'% done')
        if count_doc % (percentil-1) == 0: clear_output()
        
        ### captura info de ano, mes e id de cada documento ###
        txt_year = re.sub('.*(19\d\d).*', r'\1', txt)
        txt_month = re.sub('.*19\d\d\.(\d\d).*', r'\1', txt)
        txt_id = re.sub('.*AAS_mre_(.*).txt', r'\1', txt)

        ### captura info de url ###
        txt_id_dossie = re.sub('.*AAS_mre_(.*)_doc.*', r'\1', txt)
        for dossie_with_url in dossie_list:
            if txt_id_dossie == dossie_with_url:
                list_number = dossie_list.index(dossie_with_url)
                url = url_list[list_number]
                
        with open(txt, 'r', encoding='ISO-8859-1') as f: #with open(txt, 'r', encoding='utf-8') as f: 
            txt_body = f.read()
           
            ### identifica idioma por documento ###
            texto_pt = texto_en = texto_es = texto_fr = txt_body
            texto_split = re.split('\.|\?|\:', txt_body)
            #print(texto_split)
            count = pt_count = en_count = es_count = fr_count = de_count = lang_count = total_count = 0
            for phrase in texto_split:
                total_count += 1
                if not re.search('[^\W\d_]', phrase): continue
                try: 
                    language = detect(phrase)
                except: 
                    print(txt)
                    print('PHRASE', phrase, 'LANGUAGE', language)
                total_count += 1
                if language == 'pt':
                    pt_count += 1
                if language == 'en':
                    en_count += 1
                if language == 'es':
                    es_count += 1
                if language == 'fr':
                    fr_count += 1
                if language == 'de':
                    de_count += 1
            lang_count = pt_count + en_count + es_count + fr_count + de_count        
            if lang_count/total_count < 0.08: 
                #print('not readable', txt)
                not_readable.append(txt)
                continue
            # obs: provavelmente sobrarão documentos com lang_class = 0. Manter sem identificação de idioma?
            if de_count/total_count > 0.1: 
                lang_class = 5
            if fr_count/total_count > 0.1: 
                lang_class = 4
            if es_count/total_count > 0.1: 
                lang_class = 3
            if en_count/total_count > 0.1: 
                lang_class = 2
            if pt_count/total_count > 0.1: 
                lang_class = 1
            
            ### insere dados na base mysql ###
            query = "INSERT INTO doc VALUES (%s, %s, %s, %s, %s, %s, %s)"
            cur.execute(query, (txt_id, txt_body, txt_year, txt_month, url, lang_class, temp))

    #cur.execute("SELECT * FROM doc");
    #rows = cur.fetchall()
    #print(rows)

1 % done
/home/rsouza/Documentos/text-learning-tools/textfiles-corrected-regrouped/AAS_mre_d_1974.04.23_doc_X-9.txt
PHRASE  '7ª"  LANGUAGE fr
